## LLM migration driven by data-aware prompt optimization - RAG-QA task

Example notebook 
- model migration from Claude 3 Haiku to Nova-Lite for a question-answer task
- optimization by Bedrock APO and data-aware optimization after migration
- evaluation metric: semantic similarity  

### Install python packages requested by benchmarking

If you have not install the requested python libraries, uncomment the following command to run the installation.

In [ ]:
#!pip install -r ../peccyben/requirements.txt

### Import libraries

In [ ]:
import sys, os, time
import pandas as pd
from sklearn.model_selection import train_test_split
import json, copy
import warnings 

sys.path.insert(0, '../')

from peccymig.migration import Prompt_Opt_Template_Gen, eval_metric_ss
from peccymig.migration import model_initialize, model_inference, data_aware_optimization, update_prompt_catalog
from peccyben.qaragtask import QA_Ben, QA_Create_VDB 
from peccyben.promptcatalog import Prompt_Template_Gen

In [ ]:
import boto3
import botocore
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

boto3_bedrock = boto3.client(service_name="bedrock", region_name="us-east-1")
boto3_bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")

# Create embedding
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0", client=boto3_bedrock_runtime)

### Data preparation

Split training and testing data for prompt optimizer

In [ ]:
# Load dataset
INPUT_FILE = 'cuad_test_50p.csv'
df_input = pd.read_csv('./data/'+INPUT_FILE, encoding='utf8')

In [ ]:
df_input_train, df_input_test = train_test_split(df_input, test_size=0.2, random_state=24)

df_input_train = df_input_train.reset_index()
df_input_test = df_input_test.reset_index()

### Evaluation before migration

Baselining the evaluation metrics for the model before migration

* **BENCH_KEY**: a unique keyname for your benchmarking in this round 
* **S3_BUCKET**: the S3 buckt you created for the benchmarking    
* **TASK_FOLDER**: the task folder you created under the S3 bucket   
* **INPUT_FILE**: the file name of the dataset you prepared for benchmarking
* **DOC_FILE**: the document to query 
* **VDB_NAME**: the vector db name for RAG pipeline   
* **METRICS_LIST**: the metrics we provide for the question-answer task    
* **BEDROCK_REGION**: the AWS region that the model benchmarking runs on Bedrock
* **COST_FILE**: the price file used for calculating model inference cost    

In [ ]:
BENCH_KEY = 'PeccyMig_202503_test'
S3_BUCKET = 'genai-sdo-llm-ben-20240310'
TASK_FOLDER = 'ben-qa'
INPUT_FILE = 'cuad_10.csv'
DOC_FILE = 'PACIRA_PHARMACEUTICALS_AGREEMENT.PDF'
VDB_NAME = "loha"
METRICS_LIST = ['Inference_Time','Input_Token','Output_Token','Throughput','RougeL-sum','Semantic_Similarity','BERT-F1','Toxicity',
                                            'Answer_Correctness','Answer_similarity','Answer_Relevancy','Context_Recall','Context_Precision','Cost','Cache_Input_Token','Cache_Output_Token']
BEDROCK_REGION = 'us-east-1'
COST_FILE = 'bedrock_od_public.csv'

Results_qa = pd.DataFrame()
Results_qa = Results_qa.assign(metric_name=METRICS_LIST) 
Results_qa = Results_qa.set_index('metric_name')

#### Create a vector DB for RAG pipeline

You can skip this step if you already created a vector db in the previous round of benchmarking

* Embedding model: here we use Titan text embedding model on Bedrock
* Set the chunk size and overlap size for your retriever embedding
* Invoke **QA_Create_VDB** to create a faiss vector store

In [ ]:
# Set chunk size and create the vector db
chunk_size = 1000
chunk_overlap = 100

QA_Create_VDB(chunk_size,chunk_overlap,bedrock_embeddings,VDB_NAME,S3_BUCKET,DOC_FILE,TASK_FOLDER)

Create DSPy dataset from DataFrame for classification task 

In [ ]:
import dspy

def create_qa_dataset(prompt_template,df):
    """
    Create DSPy dataset from DataFrame for question-answer task 
    Args: 
        prompt_template: the prompt template for the question-answer task defined by user
        df: input dataset in DataFrame format
    Return: DSPy dataset
    """        
    
    dspy_dataset = []

    # create embedding
    bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0", client=boto3_bedrock_runtime)
    
    # load vectordb 
    vectorstore_faiss_general = FAISS.load_local(VDB_NAME, bedrock_embeddings,allow_dangerous_deserialization=True)
    wrapper_store_faiss_general = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_general)
    
    # vectordb search
    SEARCH_K = 5
    retriever = vectorstore_faiss_general.as_retriever(search_kwargs={
        'k': SEARCH_K,
    })
    
    for i in range(len(df)):
        docs = retriever.get_relevant_documents(df['instruction'][i])
        contexts = []
        for k in range(len(docs)):
            context_text = docs[k].page_content
            contexts.append(context_text)
        
        example = dspy.Example(
            question = prompt_template.format(question=df['instruction'][i],context=contexts),
            answer = df['response'][i]
        ).with_inputs("question")

        dspy_dataset.append(example)
        #print(len(dspy_dataset))
        
    return dspy_dataset

#### Task specific setting

* Configure your **prompt** in the prompt catalog (prompt_catalog.json), and configure the prompt_catalog_id
* Set the **LLM hyperparameter** in model_kwargs. For the models on Bedrock, refer to [inferenceConfig](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference-call.html)
* Set **LLM-judge model** for RAGAS

In [ ]:
prompt_catalog_id = "qa-1"

model_kwargs = {
        'maxTokens': 1024, 
        'topP': 0.9, 
        'temperature': 0
}   
judge_model_id = "mistral.mistral-large-2402-v1:0"

#### Specify the model and other settings for benchmarking

Invoke **QA_Ben** function to conduct the benchmarking for one selected model

* **method**: set "Bedrock" for the models on Bedrock
* **region**: configured in the previous step
* **model_id**: specify the Model ID for the model endpoint
* **judge_model_id**: specify LLM-judge model for RAGAS
* **model_kwargs**: configured in previous step
* **prompt_template**: prompt template based on the prompt configured in previous step
* **s3_bucket**: configured in previous step
* **file_name**: configured in previous step
* **BENCH_KEY**: configured in previous step
* **task_folder**: configured in previous step
* **cost_key**: set "public" when using AWS public pricing to calculate the cost
* **save_id**: the model name displayed in the report 
* **SLEEP_SEC**: you can configure "sleep and retry" when throtting, for example, set SLEEP_SEC = 10 to wait for 10 seconds between each inference
* **SAMPLE_LEN**: you can configure the number of samples for inference
* **PP_TIME**: if you want to run model inference for multiple rounds, set the number of rounds here.  
* **cacheconf**: set "default" to enable Bedrock Prompt Caching in the inference, "None" to disable
* **latencyOpt**: set "optimized" to enable Bedrock Latency Optimized Inference, "None" to disable

In [ ]:
BEFORE_MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
before_save_id = 'Haiku-3'

In [ ]:
prompt_template = Prompt_Template_Gen(BEFORE_MODEL_ID, prompt_catalog_id)
print(prompt_template)

In [ ]:
Results_qa[before_save_id] = QA_Ben(method="Bedrock",
                                   region=BEDROCK_REGION,
                                   model_id=BEFORE_MODEL_ID,
                                   judge_model_id=judge_model_id,
                                   model_kwargs=model_kwargs,
                                   prompt_template=prompt_template,
                                   vdb_name=VDB_NAME,
                                   s3_bucket=S3_BUCKET,
                                   file_name=INPUT_FILE,
                                   BENCH_KEY=BENCH_KEY,
                                   task_folder=TASK_FOLDER,
                                   cost_key=COST_FILE,
                                   save_id=before_save_id,
                                   SLEEP_SEC=20,
                                   SAMPLE_LEN=5,          # len(df_input_test),,
                                   PP_TIME=1,
                                   cacheconf="None",latencyOpt="None")
Results_qa

### Migration starts here ...

* **MODEL_ID**: specify the target model for migration
* **opt_model_id**: Bedrock model ID for optimizer
* **OPT_ITERATION**: specify the iteration number for target model prompt optimization 

In [ ]:
MODEL_ID = model_id = 'us.amazon.nova-lite-v1:0' 
opt_model_id = 'amazon.nova-lite-v1:0'
OPT_ITERATION = 5

#### Step 1: Bedrock APO 

Run Bedrock APO to get optimized prompt for the target model

In [ ]:
#apo_prompt_template = Prompt_Opt_Template_Gen('us-west-2', opt_model_id, prompt_template)

# Alternatively you can manually create the prompt following the Nova model prompt best practices
apo_prompt_template = """
## General Instructions
You are an intelligent question answering bot. Your role is to provide concise and accurate answers based on the given context and question. Carefully read and understand the provided context, then formulate your response to directly answer the question without any preamble or additional explanations.

- If you do not have enough information to determine the answer, simply respond with "I don't know" within the <results> tags.
- Do not make up answers. Only provide factual information derived from the given context.
- Be brief and concise in your response.

## Context
{{context}}

## Question
{{question}}

## Response Format
<results>
[Your answer here]
</results>
"""

# Check the prompt output and manually update the prompt language and formatting
apo_prompt_template = apo_prompt_template.replace("{{context}}", "{context}")
apo_prompt_template = apo_prompt_template.replace("{{question}}", "{question}")

print(apo_prompt_template)

Evaluate the target model performance using **QA_Ben** function

In [ ]:
save_id = 'nova-lite(apo)'
Results_qa[save_id] = QA_Ben(method="Bedrock",
                                   region=BEDROCK_REGION,
                                   model_id=MODEL_ID,
                                   judge_model_id=judge_model_id,
                                   model_kwargs=model_kwargs,
                                   prompt_template=prompt_template,
                                   vdb_name=VDB_NAME,
                                   s3_bucket=S3_BUCKET,
                                   file_name=INPUT_FILE,
                                   BENCH_KEY=BENCH_KEY,
                                   task_folder=TASK_FOLDER,
                                   cost_key=COST_FILE,
                                   save_id=save_id,
                                   SLEEP_SEC=20,
                                   SAMPLE_LEN=5,          # len(df_input_test),,
                                   PP_TIME=1,
                                   cacheconf="None",latencyOpt="None")
Results_qa

#### Step 2: data-aware optimization

Prepare training dataset and model initialization

In [ ]:
train_set = create_qa_dataset(apo_prompt_template,df_input_train)
model = model_initialize(MODEL_ID)

Data-aware optimization process: you can specify the following parameters for the optimizer 

* **model**: the initialized target model in the previous step
* **train_set**: training dataset prepared in the previous step 
* **eval_metric_accuracy**: evaluation function 
* **num_candidates**: number of prompt candidate to generate and evaluate by the optimizer
* **num_trials**: number of optimization trials to run by the optimizer
* **minibatch_size**: optimize and evaluate prompt candidates over minibatch (subset of the full training set) 
* **minibatch_full_eval_steps**: every number of steps to run full evaluation on the top averaging set of prompt candidates

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')

    for k in range(OPT_ITERATION):
        optimized_model = data_aware_optimization(model,train_set,eval_metric_ss,
                                                num_candidates = 5,
                                                num_trials = 7,
                                                minibatch_size = 20,
                                                minibatch_full_eval_steps = 7)
    

        print("========== Optimized prompt instruction ===============")
        print(optimized_model.prog.predict.signature.instructions)     # optimized_model.prog.predict.signature.instructions
        print("=======================================================")
        print("retry ",k,"...") 
        if k< OPT_ITERATION-1:
            time.sleep(60)        

Add the data-aware optimized prompt template to the prompt catalog 

In [ ]:
# Add the data-aware optimized prompt template to the prompt catalog 
update_prompt_catalog(prompt_catalog_id,optimized_model)

dao_prompt_template = Prompt_Template_Gen(MODEL_ID, prompt_catalog_id+'-dao')
print(dao_prompt_template)

Evaluate the target model performance using **QA_Ben** function. Based on the performance, you can go back to step 2 to re-run the optimization by using different training set and/or different optimizer parameters. 

In [ ]:
save_id = 'nova-micro(data-aware-opt)'
Results_qa[save_id] = QA_Ben(method="Bedrock",
                                   region=BEDROCK_REGION,
                                   model_id=MODEL_ID,
                                   judge_model_id=judge_model_id,
                                   model_kwargs=model_kwargs,
                                   prompt_template=dao_prompt_template,
                                   vdb_name=VDB_NAME,
                                   s3_bucket=S3_BUCKET,
                                   file_name=INPUT_FILE,
                                   BENCH_KEY=BENCH_KEY,
                                   task_folder=TASK_FOLDER,
                                   cost_key=COST_FILE,
                                   save_id=save_id,
                                   SLEEP_SEC=20,
                                   SAMPLE_LEN=5,          # len(df_input_test),,
                                   PP_TIME=1,
                                   cacheconf="None",latencyOpt="None")
Results_qa

#### Select your prompt for the target model based on the performance evaluation 

In [ ]:
print("Optimized prompt from step 1")
print(apo_prompt_template)
print("Optimized prompt from step 2")
print(dao_prompt_template)